In [ ]:
import pandas as pd
import numpy as np

from bikewheelcalc import BicycleWheel, Hub, Rim
import bikewheellib as bl

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline

In [ ]:
# Set default colors
default_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                  '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                  '#bcbd22', '#17becf']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=default_colors)

# Default markers
mrk = ['o', '*', 'd', 's', '^', 'p', 'v']

## Load-displacement curves

In [ ]:
kN = 0.00444822 # Conversion from lbs to kN

pd1 = pd.read_csv('../data/rad_buckling_experiments/BR17125839_Pd.csv', header=0)
pd2 = pd.read_csv('../data/rad_buckling_experiments/BR17282956_Pd.csv', header=0)
pd3 = pd.read_csv('../data/rad_buckling_experiments/BR17282957_Pd.csv', header=0)

pd1['FORCE_kN'] = pd1['FORCE']*kN
pd2['FORCE_kN'] = pd2['FORCE']*kN
pd3['FORCE_kN'] = pd3['FORCE']*kN

rad_results = pd.DataFrame({'LAT_LOAD': [58., 58., 220.],
                            'RAD_STIFF': 0.,
                            'RAD_STIFF_err': 0.})

In [ ]:
def trim_rad(pdn, n_trim):
    'Trim and shift non-linear settling portion of load-displacement curve.'
    
    pdn = pdn.drop(index=range(n_trim))
    
    pf = np.polyfit(pdn['EXT_RAD'][:40], pdn['FORCE'][:40], 1)
    
    v_offset = -pf[1]/pf[0]
    
    pdn['EXT_RAD'] = pdn['EXT_RAD'] - v_offset
    
    return pdn

# Trim non-linear settling portion at the beginning
pd1 = trim_rad(pd1, 5)
pd2 = trim_rad(pd2, 0)
pd3 = trim_rad(pd3, 5)

In [ ]:
cm = sns.color_palette('Blues', 3)

with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(ncols=2, figsize=(6.5, 3.25))

    ax[0].plot(pd1['EXT_RAD'], pd1['FORCE_kN'], '.:', color=cm[2])
    ax[0].plot(pd2['EXT_RAD'], pd2['FORCE_kN'], '.:', color=cm[1])
    ax[0].plot(pd3['EXT_RAD'], pd3['FORCE_kN'], 'C3.:')
    
    ax[0].set_ylim([0., 5.])
    ax[0].set_xlim([0., 12.])
    
    ax[0].set_xlabel('Radial displacement [mm]')
    ax[0].set_ylabel('Radial load [kN]')
    
    
    ax[1].plot(pd1['EXT_LAT'], pd1['EXT_RAD'], '.:', color=cm[2])
    ax[1].plot(pd2['EXT_LAT'], pd2['EXT_RAD'], '.:', color=cm[1])
    ax[1].plot(pd3['EXT_LAT'], pd3['EXT_RAD'], 'C3.:')
    
    ax[1].set_xlim([-5., 42.])
    ax[1].set_ylim([0., 12.])
    
    ax[1].set_xlabel('Lateral displacement [mm]')
    ax[1].set_ylabel('Radial displacement [mm]')
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_NU_experiments.svg')

### Peak loads and strength difference

In [ ]:
print 'Wheel 1: {:.2f} kN ({:.1f} lbs)'.format(pd1['FORCE_kN'].max(), pd1['FORCE'].max())
print 'Wheel 2: {:.2f} kN ({:.1f} lbs)'.format(pd2['FORCE_kN'].max(), pd2['FORCE'].max())
print 'Wheel 3: {:.2f} kN ({:.1f} lbs)'.format(pd3['FORCE_kN'].max(), pd3['FORCE'].max())

print '\nStrength difference:'
print '{:.2f} kN ({:.1f} lbs)'.format(pd1['FORCE_kN'].max() - pd3['FORCE_kN'].max(),
                                      pd1['FORCE'].max() - pd3['FORCE'].max())

### Spoke buckling load (onset of nonlinearity)

In [ ]:
for i, pdn in enumerate([pd1, pd2, pd3]):
    
    # Linear fit to initial slope
    lf = np.polyfit(pdn['EXT_RAD'][:40], pdn['FORCE'][:40], 1)
    
    # Percentage deviation from linear slope
    P_diff = (lf[0]*pdn['EXT_RAD'] + lf[1] - pdn['FORCE']) / pdn['FORCE']
    
    # Index of nonlinearity point
    i_nl = P_diff[(P_diff > 0.05) & (pdn['FORCE'] > 100)].index[0]
    
    # Calculate radial stiffness
    i_fit = int(np.floor(0.8*i_nl))
    lf2 = smf.ols('FORCE_kN ~ EXT_RAD', data=pdn.iloc[:i_fit]).fit()
    
    rad_results.loc[i, 'LOAD_NONLIN_kN'] = pdn['FORCE_kN'][i_nl]
    
    rad_results.loc[i, 'RAD_STIFF'] = lf2.params[1]
    rad_results.loc[i, 'RAD_STIFF_err'] = lf2.bse[1]
    rad_results.loc[i, 'PEAK_LOAD_kN'] = pdn['FORCE_kN'].max()
    rad_results.loc[i, 'EXT_RAD_MAX'] = pdn['EXT_RAD'].max()
    
    plt.plot(pdn['EXT_RAD'], pdn['FORCE'])
    plt.plot(pdn['EXT_RAD'][i_nl], pdn['FORCE'][i_nl], 'r*')
    
rad_results

# Post-mortem: spoke tensions and deformation

In [ ]:
pm1 = pd.read_csv('../data/rad_buckling_experiments/BR17125839_post_mortem.csv', header=0)
pm2 = pd.read_csv('../data/rad_buckling_experiments/BR17282956_post_mortem.csv', header=0)
pm3 = pd.read_csv('../data/rad_buckling_experiments/BR17282957_post_mortem.csv', header=0)

In [ ]:
def c(x, i):
    'Shift elements of x so that the ith element is in the center'
    
    return np.roll(x, len(x)/2 - i)


def plot_wheel(pm, spk_load, ax1):
    Tdiff = c((pm['T_POST'] - pm['T_PRE']).tolist(), spk_load)
    lat_def = c((pm['LAT_DEF'] - pm['LAT_DEF'].mean()).tolist(), spk_load)
    
    ax1.bar(range(1, 37, 2), Tdiff[::2], color='C0', alpha=0.8)
    ax1.bar(range(2, 37, 2), Tdiff[1::2], color='C1', alpha=0.8)
    ax1.bar(18, Tdiff[17], color='C3')
    
    ax2 = ax1.twinx()
    ax2.plot(range(1, 37), -lat_def, color='#333333')
    
    ax1.set_xticks([])
    ax1.set_ylim([-1000, 1000])
    ax2.set_ylim([-10, 10])
    
    ax1.set_yticklabels([])
    ax2.set_yticklabels([])


with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(ncols=3, figsize=(5.5, 2.25))
    
    plot_wheel(pm1, 6, ax[0])
    plot_wheel(pm2, 14, ax[1])
    plot_wheel(pm3, 14, ax[2])
    
    ax[1].set_xlabel('Spoke position')
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_NU_experiments_postmortem.pdf')

In [ ]:
for i, pm in enumerate([pm1, pm2, pm3]):
    
    rad_results.loc[i, 'T_PRE'] = pm['T_PRE'].mean()
    rad_results.loc[i, 'T_PRE_std'] = pm['T_PRE'].std()
    
    rad_results.loc[i, 'T_POST'] = pm['T_POST'].mean()
    rad_results.loc[i, 'T_POST_min'] = pm['T_POST'].min()
    rad_results.loc[i, 'T_POST_max'] = pm['T_POST'].max()
    
rad_results.transpose()

# Compare against competing failure mode model

In [ ]:
w = BicycleWheel()

# Include a small correction (3 mm on each side) to the hub width due
#   to the hub flange thickness and projection of the spoke elbow
w.hub = Hub(diam1=0.058, width1=0.025+0.001)

# CR18-700
w.rim = Rim(radius=0.304,
            area=0.538 / (2*np.pi*2700.*0.304),
            I11=25.9 / 26.0e9,  # From acoustic test
            I22=113. / 69.0e9,  # From acoustic test
            I33=266. / 69.0e9,  # From acoustic test
            Iw=0.0 / 69.0e9,
            young_mod=69.0e9,
            shear_mod=26.0e9)

# Neglect the (unknown) shear center offset
w.rim.sec_params = {'y_c': 0.0, 'y_s': 0.0}

w.lace_radial(n_spokes=36, diameter=1.83e-3, young_mod=210e9, offset=0.0)

R = w.rim.radius
EI2 = w.rim.young_mod*w.rim.I22
GJ = w.rim.shear_mod*w.rim.I11
EA = w.spokes[0].EA

# Estimate from theory
K_lat0 = bl.lateral_stiffness(w, tension=0.)

# Fitted from load-displacement curves
K_rad = bl.calc_rad_stiff(w)

# Determined by four-point bend test
Tc = bl.calc_buckling_tension(w)[0]

P_c_theor = K_lat0*R / (1 + EA/Tc * K_lat0/K_rad) / 1000.
P_c_exp = rad_results.loc[[0, 1], 'PEAK_LOAD_kN'].mean()

print 'K_lat^0     {:7.1f} N/mm'.format(K_lat0 / 1000)
print 'K_rad       {:7.1f} kN/mm'.format(K_rad / 1e6)
print 'EA_eff:     {:7.1f} kN'.format(EA/1000)
print 'Tc:         {:7.1f} N'.format(Tc)
print 'P_c theory: {:7.2f} kN'.format(P_c_theor)
print 'P_c exp:    {:7.2f} kN'.format(P_c_exp)

print ''
print 'Error: {:1.1%}'.format(np.abs(P_c_theor - P_c_exp)/P_c_exp)